In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from finitediff.grid import adapted_grid
from finitediff.tests.test_grid import g
%matplotlib inline

In [ ]:
def plot_convergence(grid_additions, cb, **kwargs):
    fig, axes = plt.subplots(1, len(grid_additions), figsize=(16, 5),
                             sharey=True, gridspec_kw={'wspace': 0})
    scores = []
    for ga, ax in zip(grid_additions, np.atleast_1d(axes)):
        grid, y = adapted_grid(0, 2, cb, ga, **kwargs)
        ax.vlines(grid, 0, 1, transform=ax.get_xaxis_transform(),
                  linestyle='--', color='k', alpha=.3, linewidth=.5)
        ax.plot(grid, cb(grid))
        between_x = grid[:-1] + np.diff(grid)/2
        between_y = y[:-1] + np.diff(y)/2
        scores.append(np.sum(np.abs(between_y - cb(between_x))))
    return np.array(scores)

In [ ]:
plot_convergence([(32,), (16, 16), (8, 8, 8, 8)], g)

In [ ]:
plot_convergence([(64,), (32, 32), (48, 12, 4)], g)

In [ ]:
blr = ((.3, .1), (.3, .1))
plot_convergence([(64,), (32, 32), (48, 12, 4)], g, blurs=blr)

In [ ]:
plot_convergence([(64,), (32, 32), (32, 24, 4, 4)], g, blurs=blr)

In [ ]:
plot_convergence([(32,)*2, (16,)*4, (8,)*8], g, blurs=blr)

In [ ]:
plot_convergence([(32,), (16,)*2, (8,)*4, (4,)*8], g, blurs=blr)

In [ ]:
plot_convergence([(48,), (24,)*2, (12,)*4, (16,)*3], g)